# 머신러닝을 활용한 텍스트 생성 모델 성능 평가

In [4]:
import pandas as pd
import random
import openai
import time


### 1. 기존 가사 데이터 수집 (자연스러운 문장 확보)

In [5]:
## CSV에서 가사 문장 읽어오기

df = pd.read_csv("../1.데이터모음/'music_data(Merge)'.csv")  # 가사 데이터 가져오기

# 컬럼 확인
print(df.columns)

# 예제: 특정 컬럼에서 문장 가져오기 (가사 컬럼)
df = df[['lyrics']].dropna()

# 문장을 한 줄씩 분리 (여러 줄의 가사를 개별 문장으로 나누기)
df['sentences'] = df['lyrics'].apply(lambda x: x.split("\n"))

# 문장 리스트로 변환
lyrics_sentences = [sent for sublist in df['sentences'] for sent in sublist]

# 데이터셋 확인
print(lyrics_sentences[:10])  

Index(['Unnamed: 0', 'title', 'artist', 'lyrics', 'release_date', 'likes'], dtype='object')
['Welcome to my world', '', 'Red , Blue , Green', '', 'My world is full of blue', '', "There's no light in your world", '', 'Red , Blue , Green', '']


### 2. 비문 데이터 자동 생성 (비자연스러운 문장 만들기) 

In [6]:
## 가사 문장을 비문으로 변환

def shuffle_words(sentence):
    words = sentence.split()
    random.shuffle(words)
    return " ".join(words)

def remove_random_word(sentence):
    words = sentence.split()
    if len(words) > 1:
        words.pop(random.randint(0, len(words)-1))
    return " ".join(words)

# 기존 가사 문장에서 비문 생성
num_samples = 1000  # 비문 데이터 개수
bad_sentences = []

for _ in range(num_samples):
    original = random.choice(lyrics_sentences)  # 원래 문장 선택
    method = random.choice([shuffle_words, remove_random_word])  # 랜덤 방식 선택
    bad_sentences.append(method(original))

# 결과 확인
print(bad_sentences[:20])  

['', 'let be the past', 'could It yeah yeah be you yeah', '하늘을 채울게', 'Twinkle little', 'Come close me', '게 말을 좋을지 하는 어디부터 무슨', '기억만 따뜻한 꺼내보자', 'It’s my', '사랑의 끝이 이 어디일지', 'baby Oh, you', '늘릴 생각밖에 씨뿌리지 종일', '내가 않을땐 더는 보이지', '아름다웠어 넌', '네 앞에 서면 말도 못 해', 'hey', '', 'Wash soul, light a brighter my day', '별들을 걸어갈래 어두운 세상 속에도 환하게 빛나는 빛을', '모습은 속에 남긴 추억 내 채 옛날']


In [34]:
print(openai.__version__)   # openai 버전 확인 0.28 이어야 함 
print(openai.__file__)  # openai 설치 경로 확인 


1.60.2
/home/wanted-1/miniconda3/envs/team5/lib/python3.11/site-packages/openai/__init__.py


### 3. ChatGPT (LLM) 활용 자동 라벨링 (자연스러움 점수 자동 부여)

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일에서 환경 변수 로드

In [ ]:


def get_sentence_score(sentence):
    prompt = f"다음 문장이 가사로서 자연스러운지 평가해줘. 0(비문)~1(자연스러움) 사이의 점수로 답해줘.\n문장: {sentence}"
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    
    # 숫자만 추출
    score_text = response["choices"][0]["message"]["content"]
    try:
        score = float(score_text.strip())  # 응답에서 숫자만 추출
    except ValueError:
        score = 0.5  # 이상한 응답이면 중립값(0.5) 적용
    return score

# 일부 가사 문장을 ChatGPT로 평가
num_samples = 200  # 평가할 문장 개수
sample_sentences = random.sample(lyrics_sentences, num_samples)  # 랜덤 샘플 선택

# 점수 부여
sentence_scores = []

for sentence in sample_sentences:
    score = get_sentence_score(sentence)
    sentence_scores.append((sentence, score))
    time.sleep(1)  # API 과부하 방지

# 결과 확인
print(sentence_scores[:10])


In [11]:
import openai

# client = openai.OpenAI(api_key="YOUR_OPENAI_API_KEY")
client = openai.OpenAI(api_key=jimin_key) # API 키 설정

# 사용 가능한 모델 리스트 확인
models = client.models.list()

for model in models.data:
    print(model.id)


AttributeError: module 'openai' has no attribute 'OpenAI'

In [12]:
# import openai

# # OpenAI API 키 설정
# openai.api_key = jimin_key

# def get_sentence_score(sentence):
#     prompt = f"다음 문장이 가사로서 자연스러운지 평가해줘. 0(비문)~1(자연스러움) 사이의 점수로 답해줘.\n문장: {sentence}"

#     response = openai.ChatCompletion.create(
#         model="gpt-3.5-turbo",  # 사용 가능한 모델 중 하나 선택
#         messages=[
#             {"role": "system", "content": "너는 한국어 가사 문장을 평가하는 AI야. 문장이 자연스러운지 0~1 사이의 점수로 평가해줘."},
#             {"role": "user", "content": prompt}
#         ],
#         temperature=0.0  # 일관된 평가를 위해 온도를 0으로 설정
#     )

#     # 응답 데이터 추출
#     score_text = response['choices'][0]['message']['content'].strip()

#     try:
#         score = float(score_text)
#     except ValueError:
#         score = 0.5  # 오류 발생 시 중립값 적용

#     return score

# # 테스트 실행
# test_sentence = "하늘 아래 너와 나, 노래를 부르네"
# print(f"문장: {test_sentence}, 점수: {get_sentence_score(test_sentence)}")

# # 오픈 api 불질러야함.. 


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

### 4. 최종 데이터셋 만들기 (CSV 저장)

In [ ]:
# 데이터 정리
data = []

# 기존 문장 : 자연스러운 문장 (1점)
for sent in lyrics_sentences:
    data.append((sent, 1.0))

# 비문 (0점)
for sent in bad_sentences:
    data.append((sent, 0.0))

# ChatGPT 평가 문장 (0~1점)
for sent, score in sentence_scores:
    data.append((sent, score))

# 데이터프레임 생성
df = pd.DataFrame(data, columns=["sentence", "score"])

# CSV 저장
df.to_csv("lyrics_sentence_dataset.csv", index=False, encoding="utf-8")
print("✅ 최종 데이터셋 저장 완료!")
